# PATSTAT PIZnet Jupyter Demo 

# Patent Applicants and Technology Distribution in Germany by Landkreis

This notebook analyzes patent applicants and technology distributions across Germany's NUTS Level 3 regions (Landkreise). Using SQL to query the PATSTAT database, we will load data into Pandas DataFrames and explore it interactively with Pygwalker. For optional geographical visualization, we will use Geopandas to create maps of applicants and technology distributions by Landkreis. 

## Setup and import the library modules

## Importing Libraries and Modules
This cell imports the necessary libraries for data processing, SQL queries, and visualization. It also includes optional imports for working with geographical data (e.g., `geopandas`).

In [3]:
# Install and import necessary libraries and modules

# Install necessary libraries if not already installed
#pip install pandas sqlalchemy pygwalker geopandas

# Import pandas libraries for data frame handling
import pandas as pd

# (Optional) Import Geopandas for mapping if needed later
import geopandas as gpd

# Import pygwalker library for vizualisation
import pygwalker as pyg

# Import sql library for easy sql execution
from sqlalchemy import create_engine, func
from sqlalchemy.sql import literal_column

# Import time library for measuring sql execution time
import time

# Import xml lib for IPC sub group labels
from lxml import etree as ET


## Importing PATSTAT specific Libraries and intantiate the ORM

Part of the new EPO Technology Intelligence Plattform is an PATSTAT module and ORM, that gives easy access to to the data here in another container of the TIP platform.

In [4]:

# Import the epo library module for PATSTAT
from epo.tipdata.patstat import PatstatClient

# Intantiate the client object / TEST or PRDOD
patstat = PatstatClient(env='TEST')
# patstat = PatstatClient(env='PROD')

# Instantiate the ORM with PATSTAT
db = patstat.orm()

# import all the models we need
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS202_APPLN_TITLE, TLS206_PERSON, TLS207_PERS_APPLN, TLS224_APPLN_CPC, TLS231_INPADOC_LEGAL_EVENT


## Test Query 1

List all granted applications filed at EPO (direct +PCT) having a year of filing = 2010.

### SQL Query 1 with ORM
This cell contains SQL queries using the ORM to analyze patent applicants and technology distributions. The results are fetched into a Pandas DataFrame for further exploration.

The resulting dataframe (df) will be printed for visual check of the working environment.

In [5]:
# Define SQL queries using ORM for analysis

# Example: Query patent applicants and technology distribution
# Test Query 1

q = db.query(TLS201_APPLN.appln_id, TLS201_APPLN.appln_auth, TLS201_APPLN.appln_nr, 
                           TLS201_APPLN.appln_kind, TLS201_APPLN.appln_filing_date).\
             filter(TLS201_APPLN.appln_filing_year == 2010,
                    TLS201_APPLN.appln_auth == 'EP',
                    TLS201_APPLN.granted == 'Y')
df = patstat.df(q)

df

,appln_id,appln_auth,appln_nr,appln_kind,appln_filing_date
0,274222610,EP,10000313,A,2010-01-14
1,274369023,EP,10000849,A,2010-01-28
2,274681480,EP,10001469,A,2010-02-12
3,274720647,EP,10001552,A,2010-02-16
4,274875659,EP,10002051,A,2010-03-01
...,...,...,...,...,...
515,364674236,EP,10860973,A,2010-12-22
516,405968954,EP,13165754,A,2010-09-20
517,448177758,EP,16150333,A,2010-08-12
518,470108142,EP,16192322,A,2010-07-16


## Test Query 2

Create a hitlist of Chinese applicants filing patents at the EPO (direct or PCT).

### SQL Query with ORM
This cell contains SQL queries using the ORM to analyze patent applicants and technology distributions. The results are fetched into a Pandas DataFrame for further exploration.

In [6]:
# Define SQL queries using ORM for analysis

# Example: Query patent applicants and technology distribution
# Test Query 2

# Start the timer
start_time = time.time()

q = db.query(
    TLS206_PERSON.psn_name, 
    TLS206_PERSON.person_ctry_code, 
    func.count(TLS201_APPLN.appln_id).label('applications_at_epo')).\
             select_from(TLS206_PERSON).\
             join(TLS207_PERS_APPLN).join(TLS201_APPLN).\
             filter(TLS206_PERSON.person_ctry_code == 'CN',
                    TLS207_PERS_APPLN.applt_seq_nr > 0,
                    TLS207_PERS_APPLN.invt_seq_nr == 0,
                    TLS201_APPLN.appln_auth == 'EP').\
             group_by(TLS206_PERSON.psn_name, TLS206_PERSON.person_ctry_code).\
             order_by(func.count(TLS201_APPLN.appln_id).desc()).limit(100)

df = patstat.df(q)

# Stop the timer
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Query execution time: {execution_time:.2f} seconds")

df

Query execution time: 0.93 seconds


,psn_name,person_ctry_code,applications_at_epo
0,BEIJING GOLDWIND SCIENCE & CREATION WINDPOWER ...,CN,100
1,XINJIANG GOLDWIND SCIENCE & TECHNOLOGY COMPANY,CN,50
2,Beijing Goldwind Science & Creation Windpower ...,CN,21
3,"Huawei Digital Power Technologies Co., Ltd.",CN,16
4,JIANGSU GOLDWIND SCIENCE & TECHNOLOGY COMPANY,CN,14
...,...,...,...
95,CBJ CONCRETE OCEAN PLATFORM ENGINEERING (SHANG...,CN,1
96,JOINTIFF,CN,1
97,GENE POWER HOLDING COMPANY,CN,1
98,GOGORO,CN,1


## ChatGPT (GPT-4o) as a Co-Developer

Initial Prompt with mtc.berlin "ChatGPT Team" with activated memory with the GPT-4o model.

>I need you to help me develop SQL statements for PATSTAT based on regional NUTS codes to identify applicants and technologies based on federal states in Germany. Please act as a co-developer and be so kind to help me with a map per federal state where we create lists for applicants and technologies distribution in level 3 NUTS. We want to visualize with pygwalker library.

>Please answer in markdown and code blocks to easily copy the results for testing in a jupyter notebook.

The following cells are co-developed in several iterations with this starting prompt above. 



In [30]:
# Query patent applicants and technology for NUTS level 3 in Germany

# Start the timer
start_time = time.time()

q = db.query(
    TLS206_PERSON.person_name,
    TLS206_PERSON.nuts.label('nuts_code'),
    TLS224_APPLN_CPC.cpc_class_symbol.label('technology_field'),  # Adding CPC class as technology field
    func.count(TLS201_APPLN.appln_id).label('appln_count')).select_from(TLS206_PERSON).\
    join(TLS207_PERS_APPLN, TLS206_PERSON.person_id == TLS207_PERS_APPLN.person_id).\
    join(TLS201_APPLN, TLS207_PERS_APPLN.appln_id == TLS201_APPLN.appln_id).\
    join(TLS224_APPLN_CPC, TLS201_APPLN.appln_id == TLS224_APPLN_CPC.appln_id).\
    filter(
        TLS206_PERSON.nuts.startswith('DE'),   # Filter for Germany NUTS code
        TLS206_PERSON.nuts_level == 3          # Limit to NUTS level 3
    ).\
    group_by(
        TLS206_PERSON.nuts,
        TLS206_PERSON.person_name,
        TLS224_APPLN_CPC.cpc_class_symbol      # Group by CPC class as well
    ).\
    order_by(TLS206_PERSON.nuts)  # Sort by nuts_code
# Convert to DataFrame for further processing and visualization

df = patstat.df(q)


# Stop the timer
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Query execution time: {execution_time:.2f} seconds")


df


Query execution time: 1.77 seconds


,person_name,nuts_code,technology_field,appln_count
0,Universität Stuttgart,DE111,F03B 15/00,1
1,Universität Stuttgart,DE111,F03B 17/061,1
2,Universität Stuttgart,DE111,F03D 7/043,2
3,Universität Stuttgart,DE111,F05B2260/845,1
4,Universität Stuttgart,DE111,F05B2270/8042,3
...,...,...,...,...
55851,"Fliegl, Helmut",DEG0K,B60D 1/015,1
55852,"Fliegl, Helmut",DEG0K,B60D 1/30,1
55853,"Fliegl, Helmut",DEG0K,B60D 1/305,1
55854,"Fliegl, Helmut",DEG0K,B60D 1/62,1


## The Visualisation of the results on a Map of Germany with Polygones

### Pyg Walker settings

1. Coordinate System: Geographic
2. Mark System: Choropleth
3. Download nuts in geojson from Eurostat (https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics) 
4. Load geojson file
5. Enter Feature ID
6. Add NUTS field to Geometry ID
7. Add Row Count to Color

In [7]:
# Execute this cell for a Tableau like visualisationin PygWalker

pyg.walk(df)


Box(children=(HTML(value='\n<div id="ifr-pyg-0006277ce40603dbDP1IecxsU28M5wQN" style="height: auto">\n    <hea…

# Query Documentation: Patent Analysis by NUTS Level 3

## Purpose:
This query retrieves detailed information about patent applications in Germany, categorized by NUTS Level 3 regions (Landkreise). It is designed to provide insights into:
- Applicants (`person_name`) operating in specific regions.
- The technologies they focus on (`technology_field`), identified using CPC classifications.
- Filing trends over time (`filing_year`).
- Patent status, specifically whether the patent has been granted (`granted`).

## Key Features:
1. **Regional Insights**:
   - Leverages NUTS Level 3 codes to localize applicants within Germany.
   - Extracts NUTS Level 1 (Federal State) codes for higher-level aggregation.

2. **Technological Focus**:
   - Links patents to their Cooperative Patent Classification (CPC) codes, enabling analysis of specific technology fields.

3. **Filing Year Trends**:
   - Includes the filing year to track application trends over time.

4. **Grant Status**:
   - Determines whether patents in the dataset have been granted.

5. **Execution Timing**:
   - Measures and reports the query execution time to provide transparency on performance.

## Data Sources:
- **Applicants**: `TLS206_PERSON`
- **Patent Applications**: `TLS201_APPLN`
- **Technology Classification**: `TLS224_APPLN_CPC`
- **Geographic Information**: NUTS codes from `TLS206_PERSON`

## Intended Audience:
This query is specifically designed for patent information professionals seeking actionable insights into:
- Regional patent activity.
- Applicant profiles and their focus areas.
- Trends in patent filings and grants across Germany.

## How to Use:
1. Execute the query in a PATSTAT-connected environment.
2. Use the resulting DataFrame (`df`) for:
   - Exploratory data analysis.
   - Visualization with tools like Pygwalker.
   - Further filtering or aggregation based on your needs.

In [38]:
# Define SQL queries using ORM for analysis
# Example: Query patent applicants and technology distribution

# Start the timer
start_time = time.time()

q = db.query(
    TLS206_PERSON.person_name,
    TLS206_PERSON.nuts.label('nuts_code'),
    literal_column("SUBSTR(nuts, 1, 3)").label('federal_state_code'),  # Federal state (NUTS Level 1)
    TLS224_APPLN_CPC.cpc_class_symbol.label('technology_field'),       # Technology field
    literal_column("SUBSTR(cpc_class_symbol, 1, 4)").label('cpc_mainclass'),
    TLS201_APPLN.appln_filing_year.label('filing_year'),               # Filing year
    TLS201_APPLN.granted.label('granted'),                             # Grant status
    func.count(TLS201_APPLN.appln_id).label('appln_count')             # Application count
).select_from(TLS206_PERSON).\
    join(TLS207_PERS_APPLN, TLS206_PERSON.person_id == TLS207_PERS_APPLN.person_id).\
    join(TLS201_APPLN, TLS207_PERS_APPLN.appln_id == TLS201_APPLN.appln_id).\
    join(TLS224_APPLN_CPC, TLS201_APPLN.appln_id == TLS224_APPLN_CPC.appln_id).\
    filter(
        TLS206_PERSON.nuts.startswith('DE'),   # Filter for Germany NUTS code
        TLS206_PERSON.nuts_level == 3          # Limit to NUTS level 3
    ).\
    group_by(
        TLS201_APPLN.appln_filing_year,        # Group by filing year
        TLS206_PERSON.nuts,                    # Group by NUTS Level 3 code
        literal_column("SUBSTR(nuts, 1, 3)"),  # Group by federal state code
        TLS224_APPLN_CPC.cpc_class_symbol,     # Group by technology field
        TLS206_PERSON.person_name,             # Group by person name
        TLS201_APPLN.granted                   # Group by grant status
    ).\
    order_by(TLS206_PERSON.nuts) #, TLS201_APPLN.appln_filing_year)

# Execute the query
df = patstat.df(q)

# Stop the timer
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Query execution time: {execution_time:.2f} seconds")


# Display the first few rows of the DataFrame
df


Query execution time: 4.57 seconds


,person_name,nuts_code,federal_state_code,technology_field,cpc_mainclass,filing_year,granted,appln_count
0,Universität Stuttgart,DE111,DE1,F03B 15/00,F03B,2019,Y,1
1,Universität Stuttgart,DE111,DE1,F03B 17/061,F03B,2019,Y,1
2,Universität Stuttgart,DE111,DE1,F03D 7/043,F03D,2019,Y,1
3,Universität Stuttgart,DE111,DE1,F05B2260/845,F05B,2019,Y,1
4,Universität Stuttgart,DE111,DE1,F05B2270/8042,F05B,2019,Y,1
...,...,...,...,...,...,...,...,...
71507,"Fliegl, Helmut",DEG0K,DEG,B60D 1/015,B60D,2014,Y,1
71508,"Fliegl, Helmut",DEG0K,DEG,B60D 1/30,B60D,2014,Y,1
71509,"Fliegl, Helmut",DEG0K,DEG,B60D 1/305,B60D,2014,Y,1
71510,"Fliegl, Helmut",DEG0K,DEG,B60D 1/62,B60D,2014,Y,1


In [53]:
import pandas as pd

# Load the CSV file
nuts_mapping = pd.read_csv('./nuts_mapping.csv', delimiter=',')

# Create separate mappings for federal states and districts
federal_state_mapping = nuts_mapping[nuts_mapping['LEVEL'] == 1].set_index('NUTS_ID')['NAME_LATIN'].to_dict()
landkreis_mapping = nuts_mapping[nuts_mapping['LEVEL'] == 3].set_index('NUTS_ID')['NAME_LATIN'].to_dict()

# Map federal states (NUTS Level 1)
df["federal_state_name"] = df["nuts_code"].str[:3].map(federal_state_mapping)

# Map Landkreise (NUTS Level 3)
df["landkreis_name"] = df["nuts_code"].map(landkreis_mapping)

# Display the first few rows of the DataFrame
print(df.head())


             person_name nuts_code federal_state_code technology_field  \
0  Universität Stuttgart     DE111                DE1      F03B  15/00   
1  Universität Stuttgart     DE111                DE1     F03B  17/061   
2  Universität Stuttgart     DE111                DE1     F03D   7/043   
3  Universität Stuttgart     DE111                DE1     F05B2260/845   
4  Universität Stuttgart     DE111                DE1    F05B2270/8042   

  cpc_mainclass  filing_year granted  appln_count federal_state_name  \
0          F03B         2019       Y            1  Baden-Württemberg   
1          F03B         2019       Y            1  Baden-Württemberg   
2          F03D         2019       Y            1  Baden-Württemberg   
3          F05B         2019       Y            1  Baden-Württemberg   
4          F05B         2019       Y            1  Baden-Württemberg   

               cpc_mainclass_title               landkreis_name  
0  MACHINES OR ENGINES FOR LIQUIDS  Stuttgart, Landeshau

## Load and parse the WIPO xml scheme 

The xml contains the IPC symbols and their respective titles. We iterate through the tree, to extract a list of the 646 Sub Group Titles. 
Then we add another column to have the CPC/IPC titel in the data frame for vizualisation. 

In [10]:

# File path to the IPC XML
filename = "./EN_ipc_scheme_20210101.xml"

# Define the namespace and parser
ipc_namespace = '{http://www.wipo.int/classifications/ipc/masterfiles}'
ipcEntry = f"{ipc_namespace}ipcEntry"
text_body = f"{ipc_namespace}textBody"
title_part = f"{ipc_namespace}titlePart"
text = f"{ipc_namespace}text"
parser = ET.XMLParser(remove_blank_text=True)

# Parse the XML file
tree = ET.parse(filename, parser=parser)
root = tree.getroot()

# Initialize dictionary for sub-class mapping
sub_class_mapping = {}

# Start measuring time
start = time.time()

# Iterate through the XML to extract sub-class information
for element in root.iter(ipcEntry):
    if element.attrib.get("kind") == "u":  # Focus on sub-classes
        symbol = element.attrib.get("symbol")  # Extract sub-class symbol

        # Locate the title text within the nested structure
        text_element = element.find(f".//{text_body}//{title_part}//{text}")
        title = text_element.text.strip() if text_element is not None else "No Title"

        sub_class_mapping[symbol] = title

# Print execution time
print(f"Extracted {len(sub_class_mapping)} sub-classes in {(time.time() - start) * 1000:.0f} ms.")

# Print a sample of the extracted data
#for symbol, title in list(sub_class_mapping.items())[:20]:
#    print(f"{symbol}: {title}")

# execute the mapping
df["cpc_mainclass_title"] = df["cpc_mainclass"].map(sub_class_mapping)

# Display the first few rows of the DataFrame
df

Extracted 646 sub-classes in 51 ms.


KeyError: 'cpc_mainclass'

## The preparation of the datafram is completed, now we vizualise the results.

In [9]:
# Start the viusalization with pygwalker
pyg.walk(df)

Box(children=(HTML(value='\n<div id="ifr-pyg-0006277d23d4395ckW3E8gFq6mBDlsZf" style="height: auto">\n    <hea…